In [174]:
from bs4 import BeautifulSoup
import urllib2
import string
import sqlite3
from functools import reduce
import numpy as np
import pandas as pd

In [175]:
CURRENT_SEASON = "2018"
NUM_YEARS = 20
MAX_CAREER_LENGTH = 26

In [345]:
def pro_football_focus_helper(url, columns):
    index = 0
    ls = []
    while(True):
        page = urllib2.urlopen(url + "&offset=" + str(index))
        soup = BeautifulSoup(page, 'lxml')
        table_div = soup.find("div", {"id" : "div_results"})
        
        try:
            body = table_div.find("tbody")
        except:
            return ls
        
        rows = body.findAll("tr", {"class": None})

        [ls.append(i) for i in map(lambda x: [x.find("td", {"data-stat": column}).text for column in columns], rows)]
        index += 100

def get_all_players(yard_min):
    url = "https://www.pro-football-reference.com/play-index/psl_finder.cgi?request=1&match=total&draft=1"
    url += "&year_min=" + str(int(CURRENT_SEASON) - NUM_YEARS + 1) + "&year_max=" + CURRENT_SEASON + "&season_start=1&season_end=-1&pos%5B%5D=wr&draft_year_min=1936"
    url += "&draft_year_max=2019&draft_slot_min=1&draft_slot_max=500&draft_pick_in_round=pick_overall&conference=any"
    url += "&draft_pos%5B%5D=wr&c1stat=rec_yds&c1comp=gt&c1val=" + str(yard_min) + "&c5val=1.0&order_by=rec_yds"
    
    return map(lambda x: x[0], pro_football_focus_helper(url, ["player"]))
        
def get_all_seasons(yard_min): 
    url = "https://www.pro-football-reference.com/play-index/psl_finder.cgi?request=1&match=single"
    url += "&year_min=" + str(int(CURRENT_SEASON) - NUM_YEARS - MAX_CAREER_LENGTH + 1) + "&year_max=" + CURRENT_SEASON + "&season_start=1&season_end=-1&pos%5B%5D=wr&draft_year_min=1936"
    url += "&draft_year_max=2019&draft_slot_min=1&draft_slot_max=500&draft_pick_in_round=pick_overall&conference=any"
    url += "&draft_pos%5B%5D=wr&c1stat=rec_yds&c1comp=gt&c1val=" + str(yard_min) + "&c5val=1.0&order_by=rec_yds"
    
    url2 = "https://www.pro-football-reference.com/play-index/psl_finder.cgi?request=1&match=combined"
    url2 += "&year_min=" + str(int(CURRENT_SEASON) - NUM_YEARS - MAX_CAREER_LENGTH + 1) + "&year_max=" + CURRENT_SEASON + "&season_start=1&season_end=-1&pos%5B%5D=wr&draft_year_min=1936"
    url2 += "&draft_year_max=2019&draft_slot_min=1&draft_slot_max=500&draft_pick_in_round=pick_overall&conference=any"
    url2 += "&draft_pos%5B%5D=wr&c1stat=rec_yds&c1comp=gt&c1val=" + str(yard_min) + "&c5val=1.0&order_by=rec_yds"
    
    
    data = pro_football_focus_helper(url, ["player", "year_id", "psl_draft_info", "rec_yds"])
    drafted_year = pro_football_focus_helper(url2, ["player", "year_min"])
    drafted_year_dict = {d[0]: d[1] for d in drafted_year}
    data = [[d[0], d[1], int(d[1]) - int(drafted_year_dict[d[0]]) + 1, 0, 1, d[3]] if d[2] == "" else [d[0], d[1], int(d[1]) - int(drafted_year_dict[d[0]]) + 1, d[2].split("-")[0], d[2].split("-")[1], d[3]] for d in data]
    
    df = pd.DataFrame(data, columns =['Name', 'Year_' + str(yard_min), 'Season_' + str(yard_min), 'Round', 'Pick', 'Receiving Yards_' + str(yard_min)], dtype = float).set_index('Name')
    df['Name'] = df.index
    return df

def first_season(yard_min):
    all_seasons = get_all_seasons(yard_min)
    all_seasons = all_seasons.sort_values(by=["Name", 'Season_' + str(yard_min)])
    prev_name = ""
    first_seasons = pd.DataFrame([], columns =['Name', 'Year_' + str(yard_min), 'Season_' + str(yard_min), 'Round', 'Pick', 'Receiving Yards_' + str(yard_min)], dtype = float)
    for idx, season in all_seasons.iterrows():
        if season['Name'] != prev_name:
            prev_name = season['Name'] 
            if season['Year_' + str(yard_min)] >= int(CURRENT_SEASON) - NUM_YEARS + 1:
                first_seasons = first_seasons.append(season)
    
    return first_seasons

In [346]:
first_seasons_900 = first_season(900)
first_seasons_1100 = first_season(1100)
        
# TODO differentiate between players with the same name (AKA Steve Smith)

In [347]:
joined = pd.concat([first_seasons_900, first_seasons_1100], axis = 1, join='inner').drop_duplicates()
joined = joined.loc[:,~joined.columns.duplicated()]

In [349]:
len(joined.loc[joined['Season_900'] == joined['Season_1100']])

53

In [348]:
joined

,Name,Year_900,Season_900,Round,Pick,Receiving Yards_900,Year_1100,Season_1100,Receiving Yards_1100
A.J. Green,A.J. Green,2011.0,1.0,1.0,4.0,1057.0,2012.0,2.0,1350.0
Adam Thielen,Adam Thielen,2016.0,3.0,0.0,1.0,967.0,2017.0,4.0,1276.0
Albert Connell,Albert Connell,1999.0,3.0,4.0,115.0,1132.0,1999.0,3.0,1132.0
Allen Robinson,Allen Robinson,2015.0,2.0,2.0,61.0,1400.0,2015.0,2.0,1400.0
Alshon Jeffery,Alshon Jeffery,2013.0,2.0,2.0,45.0,1421.0,2013.0,2.0,1421.0
Amani Toomer,Amani Toomer,1999.0,4.0,2.0,34.0,1183.0,1999.0,4.0,1183.0
Amari Cooper,Amari Cooper,2015.0,1.0,1.0,4.0,1070.0,2016.0,2.0,1153.0
Andre Johnson,Andre Johnson,2003.0,1.0,1.0,3.0,976.0,2004.0,2.0,1142.0
Anquan Boldin,Anquan Boldin,2003.0,1.0,2.0,54.0,1377.0,2003.0,1.0,1377.0
Antonio Brown,Antonio Brown,2011.0,2.0,6.0,195.0,1108.0,2011.0,2.0,1108.0
